In [1]:
import os
import gzip
import numba
import zipfile

import nibabel as nib
from scipy import ndimage
import scipy.special as sc

import numpy as np
from numba import jit
from numba import njit
import threading as tr

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

import pandas as pd
import datetime as dt
### numba warrnings ###########
from numba import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df_decision = pd.read_csv("/content/drive/MyDrive/JN/NN/COVID_detection_picture/data/new_data_decision.csv", header = None)
df_validation = pd.read_csv("/content/drive/MyDrive/JN/NN/COVID_detection_picture/data/data_validate.csv", header = None)

In [4]:
df_validation.head()

,0,1
0,0d2c2e2355ef003b59db4af6cae2bf07,0
1,7a3066790cf024972d51480e095e976c,0
2,6ff9db5460fd6d2d050f77ee2976fdca,0
3,3b070df8068fc42b35b32257d664c369,0
4,7bc7e3ba48fe95a901481d3dc294bddc,0


In [5]:
df_decision.head()

,0,1
0,55c0543ab410f3c3ce6687ebebad1821,1
1,2d5125b195fd987966f09441a207a61d,1
2,6f10da723511d7bf513878652ed07f4d,1
3,0f142314b332ce9efb8df615f5482df3,1
4,afc895f528c791d932ff0c2cfbd5ff21,1


In [6]:
t_start = dt.datetime.now()

In [7]:
import traceback
import contextlib

# Some helper code to demonstrate the kinds of errors you might encounter.
@contextlib.contextmanager
def assert_raises(error_class):
    try:
        yield
    except error_class as e:
        print('Caught expected exception \n  {}:'.format(error_class))
        traceback.print_exc(limit=2)
    except Exception as e:
        raise e
    else:
        raise Exception('Expected {} to be raised but no error was raised!'.format(error_class))

In [8]:
# # Build model.
# @tf.function(input_signature=(tf.TensorSpec(shape=[None], dtype=tf.int32),))
def create_model_():
#     model = get_model_8L_(width=128, height=128, depth = 64)
    # Loads the weights
#     loaded_model = keras.models.load_model('/content/drive/MyDrive/JN/NN/COVID_detection_picture/Keras/model/model_ex-083_acc-0_951409.h5')
    #loaded_model = keras.models.load_model('/content/drive/MyDrive/JN/NN/COVID_detection_picture/Keras/model/model_keras_5L.h5')
     loaded_model = keras.models.load_model('/content/drive/MyDrive/JN/NN/COVID_detection_picture/Keras/model/model_keras_F1_0819.h5')
    # model.load_weights(checkpoint_path)
     return loaded_model
    # model.summary()
loaded_model = create_model_()

In [9]:
def read_nifti_file(filepath):
    """Read and load volume"""
    # Read file
    scan = nib.load(filepath)
    # Get raw data
    scan = scan.get_fdata()
    return scan
def normalize(volume):
    """Normalize the volume"""
    min = -1000
    max = 400
    volume[volume < min] = min
    volume[volume > max] = max
    volume = (volume - min) / (max - min)
    volume = volume.astype("float32")
    return volume

# from numba import ndimage

# @jit(parallel=True, cache=True, )
def resize_volume(img):
    """Resize across z-axis"""
    # Set the desired depth
    # desired_depth = 64 #old
    desired_depth = 64
    desired_width = 128
    desired_height = 128
    # Get current depth
    current_depth = img.shape[-1]
    current_width = img.shape[0]
    current_height = img.shape[1]
    # Compute depth factor
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    # Rotate
    # img = ndimage.rotate(img, 90, reshape=False)
    img = np.rot90(img)
    # Resize across z-axis
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    return img


def process_scan(path):
    """Read and resize volume"""
    # Read scan
    volume = read_nifti_file(path)
    # Normalize
    volume = normalize(volume)
    # Resize width, height and depth
    volume = resize_volume(volume)
    return volume
### TESTING BLOCK #####################
# test_image = process_scan(test_image)
# test_image.shape
######################################

In [10]:
image = process_scan("/content/drive/MyDrive/JN/NN/COVID_detection_picture/MRI/health_0a38b4c71a46ce68c8b48c95b93619b2.nii")

In [11]:
loaded_model.predict(np.expand_dims(image, axis=0))

array([[0.11529317]], dtype=float32)

In [12]:
image = process_scan("/content/drive/MyDrive/JN/NN/COVID_detection_picture/MRI/sick_7c983eabf45915f3e7e9cae2aa9a5569.nii")

In [13]:
loaded_model.predict(np.expand_dims(image, axis=0))

array([[0.5004264]], dtype=float32)

In [14]:
def main(path):
    label_image = ['normal', 'abnormal']
    image = process_scan(path)
    prediction = loaded_model.predict(np.expand_dims(image, axis=0))[0];
    scores = [1 - prediction[0], prediction[0]]
    class_names = ["normal", "abnormal"]
    for score, name in zip(scores, class_names):
        print(f"This model is {100 * score:.2f} percent confident that CT scan is {name}")
    if scores[0] > scores[1]:
        return ["0", str(scores[0])]
    else:
        return ["1", str(scores[1])]

In [15]:
main("/content/drive/MyDrive/JN/NN/COVID_detection_picture/MRI/health_0a38b4c71a46ce68c8b48c95b93619b2.nii")

This model is 88.47 percent confident that CT scan is normal
This model is 11.53 percent confident that CT scan is abnormal


['0', '0.8847068250179291']

In [16]:
main("/content/drive/MyDrive/JN/NN/COVID_detection_picture/MRI/sick_7c7160149aec1ebf15b28166f5458c49.nii")

This model is 61.34 percent confident that CT scan is normal
This model is 38.66 percent confident that CT scan is abnormal


['0', '0.6134120523929596']

In [17]:

path = "/content/drive/MyDrive/JN/NN/COVID_detection_picture/README/test_mosmed/"
path

'/content/drive/MyDrive/JN/NN/COVID_detection_picture/README/test_mosmed/'

In [18]:
len(os.listdir(path))

210

In [19]:
# ### создаем энкодер для файлов gzip
# def encoder_gzip2nii_(path, output_path = None):
# #     global PROJECT_DIR
#     os.chdir(path)
#     for item in os.listdir(path):
#         # print(item)
#         if (item.find('.')) > 0:
#             # print(str(item.find('.')))
#             continue
#         else:
#             if item.endswith(".gzip"):
#                 print('Dot not find, GZIP find')
#                 input = gzip.GzipFile(item)
#                 # print(item)
#                 data = input.read()
#                 input.close()
#             elif item.endswith(".zip"):
#                 print('Dot not find, ZIP find')
#                 input = zipfile.ZipFile.extract(item)
#                 # print(item)
#                 data = input.read()
#                 input.close()
#             elif item.endswith("") and (item.find('.') == -1):
#                 print('Dot not find, GZIP find')
#                 input = gzip.GzipFile(item)
#                 # print(item)
#                 data = input.read()
#                 input.close()
#             file_output = path + item + ".nii"
#             output = open(file_output, 'wb')
#             output.write(data)
#             output.close()
#             os.remove(item)
# encoder_gzip2nii_(path)

In [20]:
### TEST MODEL # AFTER TESTING COMMENT THIS BLOCK ###
# print(os.getcwd()) # check path to project folder
# path = "/C:/Users/Admin/Documents/COVID_detection_picture/MRI/" # full path to image.nii 
# path = os.getcwd() + "/MRI/sick_7ff18f5d3de11b9ae7a9e5d651313fbd.nii" # check classification
# print(path) # check full path to image .nii
data_decision = pd.DataFrame([], columns = [])
for idx, item in enumerate(os.listdir(path)):
    if idx <= 500:
#         print(idx)
#         print()
        clf = main(path + item)
#         print(item.split(".")[:-1], clf)
        data_decision = data_decision.append({"0" : str(item.split(".")[:-1]).strip("[' ']"), "1" : clf[0]}, ignore_index = True)
#         data_decision.loc[str(item.split(".")[:-1]).strip("[' ']")] = clf[0]
#         data_decision.loc[idx] = [[str(item.split(".")[:-1]).strip("[' ']")], clf[0]]
#         print()
####################################################
data_decision.head()

This model is 90.68 percent confident that CT scan is normal
This model is 9.32 percent confident that CT scan is abnormal
This model is 61.79 percent confident that CT scan is normal
This model is 38.21 percent confident that CT scan is abnormal
This model is 86.93 percent confident that CT scan is normal
This model is 13.07 percent confident that CT scan is abnormal
This model is 86.26 percent confident that CT scan is normal
This model is 13.74 percent confident that CT scan is abnormal
This model is 63.71 percent confident that CT scan is normal
This model is 36.29 percent confident that CT scan is abnormal
This model is 95.26 percent confident that CT scan is normal
This model is 4.74 percent confident that CT scan is abnormal
This model is 76.52 percent confident that CT scan is normal
This model is 23.48 percent confident that CT scan is abnormal
This model is 80.80 percent confident that CT scan is normal
This model is 19.20 percent confident that CT scan is abnormal
This model

,0,1
0,2bdf912317531b7e04b69f1341745263,0
1,5d9dff8db2a8f5c55430cfda670ccbf0,0
2,451fb9a76ca9a9f1b91b5e22640dc620,0
3,afc895f528c791d932ff0c2cfbd5ff21,0
4,d722c5de0139536981f0d916587d63b3,0


In [21]:
data_decision.shape

(210, 2)

In [22]:
data_decision.to_csv("data_decision.csv", sep = ',', 
                     header = False,  columns = None, 
                     index = False, index_label = '0',)

In [23]:
df = pd.read_csv("data_decision.csv")
df.head()

,2bdf912317531b7e04b69f1341745263,0
0,5d9dff8db2a8f5c55430cfda670ccbf0,0
1,451fb9a76ca9a9f1b91b5e22640dc620,0
2,afc895f528c791d932ff0c2cfbd5ff21,0
3,d722c5de0139536981f0d916587d63b3,0
4,25c98bca3c60245b4b54d2e7ada4467a,0


In [24]:
import csv
with open("data_decision.csv", 'r') as f_obj:
    read = csv.reader(f_obj)
    try:
        read == None
        print('Data')
    except:
        print('Data is empty')

Data


In [25]:
t_finish = dt.datetime.now() - t_start
t_finish.seconds

390